In [58]:
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from pandas import Timestamp
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import ZeroPadding2D

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load data
data_2009 = pd.read_pickle('../data/2009_df.pkl')
data_2009['month'] = data_2009['time_counter'].apply(lambda x: x.month)
data_2009['year'] = 2009

print(data_2009.head)
# Grid dimensions
x_max, y_max = 1440, 1020

# Features and target
features = ['SST', 'SAL', 'ice_frac', 'mixed_layer_depth', 'heat_flux_down', 
            'water_flux_up', 'stress_X', 'stress_Y', 'currents_X', 'currents_Y', 
            'nav_lat', 'nav_lon', 'month']
feature_to_predict = ['fco2_pre']

# Normalization
scaler_features = MinMaxScaler()  # Use MinMaxScaler for normalization
scaler_target = MinMaxScaler()

# Normalize features
data_2009[features] = scaler_features.fit_transform(data_2009[features])
data_2009[feature_to_predict] = scaler_target.fit_transform(data_2009[feature_to_predict])

# Initialize arrays
input_array = np.full((12, x_max, y_max, len(features)), np.nan, dtype=np.float32)
target_array = np.full((12, x_max, y_max, 1), np.nan, dtype=np.float32)

# Vectorized filling of arrays
for month in range(1, 13):
    month_data = data_2009[data_2009['month'] == month]
    x_indices = month_data['x'].astype(int)
    y_indices = month_data['y'].astype(int)
    
    # Filter out-of-bound indices
    valid_indices = (x_indices < x_max) & (y_indices < y_max)
    x_indices = x_indices[valid_indices]
    y_indices = y_indices[valid_indices]
    
    # Ensure there are valid data points before assignment
    if len(x_indices) == 0 or len(y_indices) == 0:
        print(month_data.head)
        print(len(y_indices))
        print(len(x_indices))
        print(f"No valid data for month {month}. Skipping...")
        break
    
    # Map data to arrays
    input_values = month_data.loc[valid_indices, features].values
    target_values = month_data.loc[valid_indices, feature_to_predict].values
    
    # Ensure shape compatibility
    if input_values.shape[0] == len(x_indices) and target_values.shape[0] == len(x_indices):
        input_array[month - 1, x_indices, y_indices, :] = input_values
        target_array[month - 1, x_indices, y_indices, 0] = target_values.flatten()
    else:
        print(f"Shape mismatch for month {month}. Skipping...")

# Fill missing values (e.g., NaNs) if needed
input_array = np.nan_to_num(input_array, nan=0.0)
target_array = np.nan_to_num(target_array, nan=0.0)

# Save the scalers for future use
import joblib
joblib.dump(scaler_features, 'scaler_features.pkl')
joblib.dump(scaler_target, 'scaler_target.pkl')

print("Input shape:", input_array.shape)
print("Target shape:", target_array.shape)


<bound method NDFrame.head of              y     x        time_counter    nav_lat  nav_lon  \
0            0     0 2009-01-16 12:00:00 -77.010475    72.75   
1            0     0 2009-02-15 00:00:00 -77.010475    72.75   
2            0     0 2009-03-16 12:00:00 -77.010475    72.75   
3            0     0 2009-04-16 00:00:00 -77.010475    72.75   
4            0     0 2009-05-16 12:00:00 -77.010475    72.75   
...        ...   ...                 ...        ...      ...   
17667379  1020  1441 2009-08-16 12:00:00  49.995502    73.00   
17667380  1020  1441 2009-09-16 00:00:00  49.995502    73.00   
17667381  1020  1441 2009-10-16 12:00:00  49.995502    73.00   
17667382  1020  1441 2009-11-16 00:00:00  49.995502    73.00   
17667383  1020  1441 2009-12-16 12:00:00  49.995502    73.00   

               time_centered  SST  tmask          e1t          e2t  ...  \
0        2009-01-16 12:00:00  0.0      0  6248.626465  6248.626465  ...   
1        2009-02-15 00:00:00  0.0      0  6248.6264

In [121]:
# import data
# we have data from ten years in a monthly resolution and a grid with dimensions 1442x1021
data_2009 = pd.read_pickle('../data/2009_df.pkl')
# data_2010 = pd.read_pickle('../data/2010_df.pkl')
# data_2011 = pd.read_pickle('../data/2011_df.pkl')
# data_2012 = pd.read_pickle('../data/2012_df.pkl')
# data_2013 = pd.read_pickle('../data/2013_df.pkl')
# data_2014 = pd.read_pickle('../data/2014_df.pkl')
# data_2015 = pd.read_pickle('../data/2015_df.pkl')
# data_2016 = pd.read_pickle('../data/2016_df.pkl')
# data_2017 = pd.read_pickle('../data/2017_df.pkl')
# data_2018 = pd.read_pickle('../data/2018_df.pkl')

In [122]:
# data_2009 = data_2009[data_2009['tmask'] == 1]
# data_2010 = data_2010[data_2010['tmask'] == 1]
# data_2011 = data_2011[data_2011['tmask'] == 1]
# data_2012 = data_2012[data_2012['tmask'] == 1]
# data_2013 = data_2013[data_2013['tmask'] == 1]
# data_2014 = data_2014[data_2014['tmask'] == 1]
# data_2015 = data_2015[data_2015['tmask'] == 1]
# data_2016 = data_2016[data_2016['tmask'] == 1]
# data_2017 = data_2017[data_2017['tmask'] == 1]
# data_2018 = data_2018[data_2018['tmask'] == 1]

data_2009['month'] = data_2009['time_counter'].apply(lambda x: x.month)
data_2009['year'] = 2009

# data_2010['month'] = data_2010['time_counter'].apply(lambda x: x.month)
# data_2010['year'] = 2010

# data_2011['month'] = data_2011['time_counter'].apply(lambda x: x.month)
# data_2011['year'] = 2011

# data_2012['month'] = data_2012['time_counter'].apply(lambda x: x.month)
# data_2012['year'] = 2012

# data_2013['month'] = data_2013['time_counter'].apply(lambda x: x.month)
# data_2013['year'] = 2013

# data_2014['month'] = data_2014['time_counter'].apply(lambda x: x.month)
# data_2014['year'] = 2014

# data_2015['month'] = data_2015['time_counter'].apply(lambda x: x.month)
# data_2015['year'] = 2015

# data_2016['month'] = data_2016['time_counter'].apply(lambda x: x.month)
# data_2016['year'] = 2016

# data_2017['month'] = data_2017['time_counter'].apply(lambda x: x.month)
# data_2017['year'] = 2017

# data_2018['month'] = data_2018['time_counter'].apply(lambda x: x.month)
# data_2018['year'] = 2018


data_2009.head()

,y,x,time_counter,nav_lat,nav_lon,time_centered,SST,tmask,e1t,e2t,...,stress_X,stress_Y,currents_X,currents_Y,fco2_pre,fco2,co2flux_pre,co2flux,month,year
0,0,0,2009-01-16 12:00:00,-77.010475,72.75,2009-01-16 12:00:00,0.0,0,6248.626465,6248.626465,...,0.0,0.0,0.0,0.0,702044352.0,702044352.0,-0.0,-0.0,1,2009
1,0,0,2009-02-15 00:00:00,-77.010475,72.75,2009-02-15 00:00:00,0.0,0,6248.626465,6248.626465,...,0.0,0.0,0.0,0.0,702044352.0,702044352.0,-0.0,-0.0,2,2009
2,0,0,2009-03-16 12:00:00,-77.010475,72.75,2009-03-16 12:00:00,0.0,0,6248.626465,6248.626465,...,0.0,0.0,0.0,0.0,702044352.0,702044352.0,-0.0,-0.0,3,2009
3,0,0,2009-04-16 00:00:00,-77.010475,72.75,2009-04-16 00:00:00,0.0,0,6248.626465,6248.626465,...,0.0,0.0,0.0,0.0,702044352.0,702044352.0,-0.0,-0.0,4,2009
4,0,0,2009-05-16 12:00:00,-77.010475,72.75,2009-05-16 12:00:00,0.0,0,6248.626465,6248.626465,...,0.0,0.0,0.0,0.0,702044352.0,702044352.0,-0.0,-0.0,5,2009


In [126]:
data_2009 = pd.read_pickle('../data/2009_df.pkl')
data_2009['month'] = data_2009['time_counter'].apply(lambda x: x.month)
data_2009['year'] = 2009

# Determine the grid dimensions
x_max = 1440 
y_max = 1020

# List of features
features = ['SST', 'SAL', 'ice_frac', 'mixed_layer_depth', 'heat_flux_down', 
            'water_flux_up', 'stress_X', 'stress_Y', 'currents_X', 'currents_Y', 
            'nav_lat', 'nav_lon', 'month']

feature_to_predict = ['fco2_pre']

# Initialize 4D array
array = np.full((6,x_max, y_max, len(features)), np.nan)
targets = np.full((6,x_max, y_max, 1), np.nan)


for i in range(1, 13):
    data = data_2009[data_2009['month'] == i]
    for _, row in data.iterrows():
        x, y = int(row['x']), int(row['y'])
        if x < 1440 and y < 1020:
            array[i-1, x, y, :] = row[features].values
            targets[i-1, x, y, :] = row[feature_to_predict].values

In [127]:
array.shape

(6, 1440, 1020, 13)

In [128]:
def build_adapted_unet_with_fix(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)


    # Bottleneck
    b = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p4)
    b = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(b)

    # Decoder
    u4 = layers.UpSampling2D((2, 2))(b)
    u4_padded = ZeroPadding2D(((0, 0), (0, 1)))(u4)  
    u4 = layers.Concatenate()([u4_padded, c4])
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u4)
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c5)

    u3 = layers.UpSampling2D((2, 2))(c5)
    u3_padded = ZeroPadding2D(((0, 0), (0, 1)))(u3) 
    u3 = layers.Concatenate()([u3_padded, c3])
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u3)
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c6)

    u2 = layers.UpSampling2D((2, 2))(c6)
    u2 = layers.Concatenate()([u2, c2])
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c7)

    u1 = layers.UpSampling2D((2, 2))(c7)
    u1 = layers.Concatenate()([u1, c1])
    c8 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(u1)
    c8 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c8)

    outputs = layers.Conv2D(1, (1, 1), activation='linear')(c8)

    return Model(inputs, outputs)

# Build and compile the model
input_shape = (1440, 1020, 13)
model = build_adapted_unet_with_fix(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_37      │ (None, 1440,      │          0 │ -                 │
│ (InputLayer)        │ 1020, 13)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_438 (Conv2D) │ (None, 1440,      │      1,888 │ input_layer_37[0… │
│                     │ 1020, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_439 (Conv2D) │ (None, 1440,      │      2,320 │ conv2d_438[0][0]  │
│                     │ 1020, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_144   │ (None, 720, 510,  │          0 │ conv2d_439[0][0]  │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_440 (Conv2D) │ (None, 720, 510,  │      4,640 │ max_pooling2d_14… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_441 (Conv2D) │ (None, 720, 510,  │      9,248 │ conv2d_440[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_145   │ (None, 360, 255,  │          0 │ conv2d_441[0][0]  │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_442 (Conv2D) │ (None, 360, 255,  │     18,496 │ max_pooling2d_14… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_443 (Conv2D) │ (None, 360, 255,  │     36,928 │ conv2d_442[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_146   │ (None, 180, 127,  │          0 │ conv2d_443[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_444 (Conv2D) │ (None, 180, 127,  │     73,856 │ max_pooling2d_14… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_445 (Conv2D) │ (None, 180, 127,  │    147,584 │ conv2d_444[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_147   │ (None, 90, 63,    │          0 │ conv2d_445[0][0]  │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_446 (Conv2D) │ (None, 90, 63,    │    295,168 │ max_pooling2d_14… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_447 (Conv2D) │ (None, 90, 63,    │    590,080 │ conv2d_446[0][0]  │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_63    │ (None, 180, 126,  │          0 │ conv2d_447[0][0]  │
│ (UpSampling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_15   │ (None, 180, 127,  │          0 │ up_sampling2d_63

 Total params: 1,964,065 (7.49 MB)

 Trainable params: 1,964,065 (7.49 MB)

 Non-trainable params: 0 (0.00 B)

In [129]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(array, targets, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [130]:
X_train = (X_train - X_train.mean()) / X_train.std()
X_val = (X_val - X_train.mean()) / X_train.std()
X_test = (X_test - X_train.mean()) / X_train.std()

In [66]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(8, 1440, 1020, 13)
(2, 1440, 1020, 13)
(2, 1440, 1020, 13)


In [131]:
history = model.fit(
    X_train, y_train,  # Input and target data for training
    validation_data=(X_val, y_val),  # Validation data
    epochs=50,  # Number of training epochs
    batch_size=1,  # Use a small batch size due to memory constraints
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - loss: 190024385560576000.0000 - mae: 270673056.0000 - val_loss: inf - val_mae: 1144869506078736384.0000
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 190020983946477568.0000 - mae: 270671616.0000 - val_loss: inf - val_mae: 1485014830066822021120.0000
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 188672329855795200.0000 - mae: 270225952.0000 - val_loss: inf - val_mae: 352143768709893931925504.0000
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 119827405715013632.0000 - mae: 233518352.0000 - val_loss: inf - val_mae: 2533613893433446488866816.0000
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 58087525712920576.0000 - mae: 193396416.0000 - val_loss: inf - val_mae: 5067643991530056049491968.0000


In [132]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 670ms/step - loss: inf - mae: 1144869506078736384.0000
Test Loss: inf, Test MAE: 1.1448695060787364e+18
